In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, RecursiveMap, Tuple, Structural } from "recursive-set";

import { RegExp } from "./03-RegExp-2-NFA";
import { parse } from "./RegExp-Parser";


import { nfa2dfa, Char } from "./01-NFA-2-DFA";
import { RegExp2NFA } from "./03-RegExp-2-NFA";

// Minimize & Equivalence
import { minimize } from "./07-Minimize";
import { fsm_complement, findWitness, regExpEquiv } from "./09-Equivalence";

// Visualisierung
import { dfa2dot, renderLegend, renderComparisonLayout } from "./FSM-2-Dot";

const viz = await instance();

# Test the Equivalence of Regular Expressions

In this notebook, we test the equivalence algorithm implemented in `09-Equivalence.ipynb`.

In [ ]:
function test(Sigma: RecursiveSet<Char>, s1: string, s2: string): void {
    try {
        const r1: RegExp = parse(s1);
        const r2: RegExp = parse(s2);

        if (regExpEquiv(r1, r2, Sigma)) {
            console.log(`✅ Equivalent:  ${s1}  ===  ${s2}`);
        } else {
            console.log(`❌ Different:   ${s1}  !==  ${s2}`);
        }
    } catch (e) {
        console.error(`Error testing equivalence for ${s1} and ${s2}:`, e);
    }
}

We define the alphabet $\Sigma = \{a, b, c\}$ and run several test cases. These examples range from simple identities to complex structural variations.

In [ ]:
const Sigma = new RecursiveSet<Char>("a", "b", "c");

In [ ]:
test(Sigma, '(ε+a)(a+ε)*(a+ε)', 'a*');

In [ ]:
test(Sigma, "(ba)(ba)*", "b(ab)*a");

In [ ]:
test(Sigma, "(a+b+c)*(ac*b+bc*a)(a+b+c)*", "c*(a(a+c)*b+b(b+c)*a)(a+b+c)*");

In [ ]:
test(
    Sigma,
    "((c*ac*)*(c*ac*)*(c*bc*)(c*bc*)*)+((c*bc*)(c*bc*)*(c*ac*)(c*ac*)*)",
    "c*(a*(a+c)*b+b*(b+c)*a)(a+b+c)*",
);

In [ ]:
test(Sigma, "(a+b)*a(a+b)*a(a+b)*a(a+b)*", "a*b*ab*ab*ab*a*");

### Visual Comparison Pipeline

The function `testVisual` orchestrates the entire verification process. It serves as a high-level driver that combines parsing, automata construction, minimization, equivalence checking, and visualization.

**The Workflow:**

1.  **Parse & Convert:** The helper `toDFA` converts the regular expression strings into DFAs (Regex $\to$ NFA $\to$ DFA). 
2.  **Minimization:** We apply `minimize` to both automata.
    * *Theoretical Note:* Two regular expressions are equivalent **if and only if** their minimal DFAs are isomorphic (structurally identical up to state renaming). This makes the visual comparison meaningful.
3.  **Equivalence Check:** We compute the difference languages using `fsm_complement` (which supports our generic/minimized automata) and search for **witnesses** using `findWitness`.
4.  **Rendering:** Finally, we generate SVG diagrams and an HTML legend to display the two minimal machines side-by-side for manual inspection.

In [ ]:
function toDFA(s: string, Sigma: RecursiveSet<Char>) {
    return nfa2dfa(new RegExp2NFA(Sigma).toNFA(parse(s)));
}

// --- 2. Main Visual Function ---

async function testVisual(
    Sigma: RecursiveSet<Char>,
    s1: string, 
    s2: string
): Promise<void> {
    console.log(`\n🔎 Inspecting: "${s1}" vs "${s2}"`);

    try {
        // 1. Berechnung (Logik)
        // Wir reichen Sigma an den Helper weiter
        const min1 = minimize(toDFA(s1, Sigma));
        const min2 = minimize(toDFA(s2, Sigma));

        const diff1 = fsm_complement(min1, min2);
        const diff2 = fsm_complement(min2, min1);
        
        const witness1 = findWitness(diff1);
        const witness2 = findWitness(diff2);
        const equivalent = witness1 === null && witness2 === null;

        // 2. Logging
        if (equivalent) {
            console.log("✅ RESULT: Equivalent!");
            console.log("Both expressions produce isomorphic minimal DFAs.");
        } else {
            console.log("❌ RESULT: NOT Equivalent.");
            if (witness1) console.log(`   Witness 1 (in L1 \\ L2): "${witness1}"`);
            if (witness2) console.log(`   Witness 2 (in L2 \\ L1): "${witness2}"`);
        }

        // 3. Rendering (Visualisierung)
        const html = renderComparisonLayout(
            s1, 
            viz.renderString(dfa2dot(min1), { format: "svg" }), 
            renderLegend(min1),
            s2, 
            viz.renderString(dfa2dot(min2), { format: "svg" }), 
            renderLegend(min2)
        );

        display.html(html);

    } catch (e) {
        console.error(e);
    }
}

In [ ]:
 await testVisual(Sigma, "(ε+a)(a+ε)*(a+ε)", "a*"); 

In [ ]:
 await testVisual(Sigma, "(ba)(ba)*", "b(ab)*a"); 

In [ ]:
 await testVisual(

Sigma,

"(a+b+c)*(ac*b+bc*a)(a+b+c)*",

"c*(a(a+c)*b+b(b+c)*a)(a+b+c)*",

); 

In [ ]:
 await testVisual(

Sigma,

"((c*ac*)*(c*ac*)*(c*bc*)(c*bc*)*)+((c*bc*)(c*bc*)*(c*ac*)(c*ac*)*)",

"c*(a*(a+c)*b+b*(b+c)*a)(a+b+c)*",

); 

In [ ]:
await testVisual(Sigma, "(a+b)*a(a+b)*a(a+b)*a(a+b)*", "a*b*ab*ab*ab*a*"); 